In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)
import keras
import warnings # to filter out warnings

warnings.filterwarnings("ignore")


2023-10-08 18:06:48.545497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load the data from keras

In [21]:

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() 

X_train = X_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

#X_train= X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
#X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])
#y_train = y_train.reshape(y_train.shape[0],1)
#y_test = y_test.reshape(y_test.shape[0],1)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [31]:
from tensorflow.keras import models,layers
#from tensorflow.keras.models import Sequential 
#from tensorflow.keras.models import Dense 
#from tensorflow.keras import regularizers
tf.random.set_seed(1234)

# Building the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())


for _ in range(2):
    model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))  # Output layer


# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32,
                    validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")


Epoch 1/10
1875/1875 [==============================] - 65s 34ms/step - loss: 0.6942 - accuracy: 0.7599 - val_loss: 0.3961 - val_accuracy: 0.8552
Epoch 2/10
1875/1875 [==============================] - 93s 50ms/step - loss: 0.4235 - accuracy: 0.8568 - val_loss: 0.3258 - val_accuracy: 0.8881
Epoch 3/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.3632 - accuracy: 0.8767 - val_loss: 0.4161 - val_accuracy: 0.8519
Epoch 4/10
1875/1875 [==============================] - 102s 54ms/step - loss: 0.3220 - accuracy: 0.8924 - val_loss: 0.3017 - val_accuracy: 0.8895
Epoch 5/10
1875/1875 [==============================] - 100s 53ms/step - loss: 0.2981 - accuracy: 0.8992 - val_loss: 0.2711 - val_accuracy: 0.9010
Epoch 6/10
1875/1875 [==============================] - 105s 56ms/step - loss: 0.2827 - accuracy: 0.9044 - val_loss: 0.2709 - val_accuracy: 0.9015
Epoch 7/10
1875/1875 [==============================] - 107s 57ms/step - loss: 0.2647 - accuracy: 0.9119 - val_loss: 0.27

In [34]:
from tensorflow.keras import models,layers
#from tensorflow.keras.models import Sequential 
#from tensorflow.keras.models import Dense 
#from tensorflow.keras import regularizers
tf.random.set_seed(1234)
def create_model():
    # Building the CNN model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())


    for _ in range(4):
        model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer


    # Compile the model
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model


model = create_model()
# Train the model
history = model.fit(X_train, y_train, epochs=1, batch_size=32,
                    validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")



'\nmodel = create_model()\n# Train the model\nhistory = model.fit(X_train, y_train, epochs=1, batch_size=32,\n                    validation_data=(X_test, y_test))\n\n# Evaluate the model\ntest_loss, test_acc = model.evaluate(X_test, y_test)\n\n# Print the test accuracy\nprint(f"\nTest Accuracy: {test_acc * 100:.2f}%")\n'

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Wrap Keras model into scikit-learn model
model = KerasClassifier(build_fn=create_model, epochs=1, batch_size=32, verbose=1)

# 5-Fold Cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=1234)

results = cross_val_score(model, X_train, y_train, cv=kfold)

# Print the cross-validation results
print(f"Baseline Accuracy: {results.mean()*100:.2f}% (+/- {results.std()*100:.2f}%)")

375/375 [==============================] - 21s 55ms/step - loss: 0.6678 - accuracy: 0.7454
Baseline Accuracy: 66.27% (+/- 7.73%)


In [ ]:
model.summary()

In [36]:
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.layers import LeakyReLU
import itertools

# Define the grid search parameters
activation_functions = ['relu', LeakyReLU()] 
optimizers = ['adam', 'adagrad']  
batch_sizes = [16, 32, 64]  
learning_rates = [0.001, 0.0001, 0.00001] 

# Placeholder to store the results
grid_search_results = []

# Loop through the parameter combinations
for params in itertools.product(activation_functions, optimizers, batch_sizes, learning_rates):
    act_func, opt, batch, lr = params
    
    # Creating the model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())
    if act_func == 'relu':
        model.add(layers.Activation('relu'))
    else:
        model.add(LeakyReLU())
    
    for _ in range(4):
        model.add(layers.Conv2D(64, (3, 3), padding='same'))
        model.add(layers.BatchNormalization())
        if act_func == 'relu':
            model.add(layers.Activation('relu'))
        else:
            model.add(LeakyReLU())
        
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer

    if opt == 'adam':
        optimizer = Adam(learning_rate=lr)
    else:
        optimizer = Adagrad(learning_rate=lr)
        
    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=1, batch_size=batch, verbose=0)  

    # Evaluate the model
    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    
    grid_search_results.append((act_func, opt, batch, lr, test_acc))
    print(f"Act: {act_func}, Opt: {opt}, Batch: {batch}, LR: {lr}, Test Acc: {test_acc*100:.2f}%")

# Print the best result
best_params = max(grid_search_results, key=lambda x: x[4])
print(f"\nBest Parameters - Act: {best_params[0]}, Opt: {best_params[1]}, Batch: {best_params[2]}, LR: {best_params[3]}, Test Acc: {best_params[4]*100:.2f}%")


Act: relu, Opt: adam, Batch: 16, LR: 0.001, Test Acc: 87.90%
Act: relu, Opt: adam, Batch: 16, LR: 0.0001, Test Acc: 87.10%
Act: relu, Opt: adam, Batch: 16, LR: 1e-05, Test Acc: 84.54%
Act: relu, Opt: adam, Batch: 32, LR: 0.001, Test Acc: 86.19%
Act: relu, Opt: adam, Batch: 32, LR: 0.0001, Test Acc: 85.66%
Act: relu, Opt: adam, Batch: 32, LR: 1e-05, Test Acc: 81.77%
Act: relu, Opt: adam, Batch: 64, LR: 0.001, Test Acc: 87.04%
Act: relu, Opt: adam, Batch: 64, LR: 0.0001, Test Acc: 86.30%
Act: relu, Opt: adam, Batch: 64, LR: 1e-05, Test Acc: 81.82%
Act: relu, Opt: adagrad, Batch: 16, LR: 0.001, Test Acc: 86.49%
Act: relu, Opt: adagrad, Batch: 16, LR: 0.0001, Test Acc: 79.53%
Act: relu, Opt: adagrad, Batch: 16, LR: 1e-05, Test Acc: 53.61%
Act: relu, Opt: adagrad, Batch: 32, LR: 0.001, Test Acc: 85.84%
Act: relu, Opt: adagrad, Batch: 32, LR: 0.0001, Test Acc: 79.23%
Act: relu, Opt: adagrad, Batch: 32, LR: 1e-05, Test Acc: 56.35%
Act: relu, Opt: adagrad, Batch: 64, LR: 0.001, Test Acc: 85.71

In [38]:
#Data Augumentation 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(
    rotation_range=10,       # Rotate images up to 10 degrees
    width_shift_range=0.1,   # Shift images horizontally
    height_shift_range=0.1,  # Shift images vertically
    shear_range=0.1,         # Shear transformation
    zoom_range=0.1           # Zoom in on images
)

# Fit the data generator to your training data
datagen.fit(X_train)

# Rebuilding your model from part (a)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

for _ in range(4):
    model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))  # Output layer

# Compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model using the augmented data generator
# Note that steps_per_epoch should usually be total_samples / batch_size
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=10,
                    validation_data=(X_test, y_test))

# Evaluate and print test accuracy
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy after Data Augmentation: {test_acc * 100:.2f}%")


Epoch 1/10
1875/1875 [==============================] - 120s 63ms/step - loss: 1.2142 - accuracy: 0.5661 - val_loss: 0.5970 - val_accuracy: 0.7362
Epoch 2/10
1875/1875 [==============================] - 111s 59ms/step - loss: 0.8000 - accuracy: 0.6822 - val_loss: 0.5826 - val_accuracy: 0.7555
Epoch 3/10
1875/1875 [==============================] - 108s 57ms/step - loss: 0.7111 - accuracy: 0.7126 - val_loss: 0.5866 - val_accuracy: 0.7562
Epoch 4/10
1875/1875 [==============================] - 126s 67ms/step - loss: 0.6369 - accuracy: 0.7366 - val_loss: 0.5265 - val_accuracy: 0.7743
Epoch 5/10
1875/1875 [==============================] - 122s 65ms/step - loss: 0.6027 - accuracy: 0.7510 - val_loss: 0.5698 - val_accuracy: 0.7738
Epoch 6/10
1875/1875 [==============================] - 121s 64ms/step - loss: 0.5717 - accuracy: 0.7728 - val_loss: 0.4635 - val_accuracy: 0.8128
Epoch 7/10
1875/1875 [==============================] - 118s 63ms/step - loss: 0.5300 - accuracy: 0.7892 - val_loss: 0

In [40]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Load the VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Print all layer names to identify where to truncate the model
for layer in base_model.layers:
    print(layer.name)

# Truncate the model at 'block4_conv1'
base_output = base_model.get_layer('block4_conv1').output

# Add new layers on top
x = GlobalAveragePooling2D()(base_output)  # Global Average Pooling layer
x = Dense(128, activation='relu')(x)  # Fully connected layer with 128 neurons
x = Dropout(0.5)(x)  # Dropout layer for regularization
predictions = Dense(10, activation='softmax')(x)  # Output layer assuming 10 classes

# Assemble the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Print the model summary to check the architecture
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Now, the model is ready to be trained on your dataset


input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 1

In [1]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Load the VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Truncate the model at 'block4_conv1'
base_output = base_model.get_layer('block4_conv1').output

# Add new layers on top
x = GlobalAveragePooling2D()(base_output)  # Global Average Pooling layer
x = Dense(128, activation='relu')(x)  # Fully connected layer with 128 neurons
x = Dropout(0.5)(x)  # Dropout layer for regularization
predictions = Dense(10, activation='softmax')(x)  # Output layer assuming 10 classes

# Assemble the final model
final_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Print the model summary to check the architecture
final_model.summary()


2023-10-12 22:21:00.137386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 22:21:25.975821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [49]:
import os
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Model, load_model

# Function to create a residual block
def residual_block(x, filters):
    shortcut = x
    
    # Main path
    x = Conv2D(filters, (3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, (3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    
    # If the number of filters does not match, apply a 1x1 conv to the shortcut path
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1,1), strides=(1,1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)  # Optional: add BatchNormalization here as well
    
    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

# Function to create the ResNet model
def create_resnet_model():
    inputs = Input(shape=(28, 28, 1))  # Adjust the input shape according to your data
    x = Conv2D(32, (3,3), strides=(1,1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Section A
    for _ in range(3):
        x = residual_block(x, 32)
    
    # Section B
    for _ in range(3):
        x = residual_block(x, 64)
    
    # Section C
    for _ in range(3):
        x = residual_block(x, 128)
    
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)  # Adjust the size and activation function according to your needs
    outputs = Dense(10, activation='softmax')(x)  # Assuming 10 classes for classification

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model
'''
model_file = 'resnet_model.h5'

# Check if the model file exists. If it does, load it. If not, create, train, and save the model.
if os.path.exists(model_file):
    print("Loading existing model")
    model = load_model(model_file)
else:
    print("Creating and training a new model")
    model = create_resnet_model()
    # Train the model with your data
    # model.fit(x_train, y_train, epochs=..., batch_size=...)
    model.save(model_file)
'''
model = create_resnet_model()
# Print model summary
model.summary()


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_296 (Conv2D)            (None, 28, 28, 32)   320         ['input_9[0][0]']                
                                                                                                  
 batch_normalization_296 (Batch  (None, 28, 28, 32)  128         ['conv2d_296[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_43 (ReLU)                (None, 28, 28, 32)   0           ['batch_normalization_296[0

In [50]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_296 (Conv2D)            (None, 28, 28, 32)   320         ['input_9[0][0]']                
                                                                                                  
 batch_normalization_296 (Batch  (None, 28, 28, 32)  128         ['conv2d_296[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_43 (ReLU)                (None, 28, 28, 32)   0           ['batch_normalization_296[0

In [52]:
# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32,
                    validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

Epoch 1/2
1875/1875 [==============================] - 1635s 872ms/step - loss: 0.4822 - accuracy: 0.8249 - val_loss: 0.6646 - val_accuracy: 0.7855
Epoch 2/2
313/313 [==============================] - 70s 224ms/step - loss: 0.4575 - accuracy: 0.8475

Test Accuracy: 84.75%
